# Homework 5

## Before reflection
Think about how people of all abilities may interact with visualizations and the internet. What accessibility challenges are you already aware of? What specific limitations or disabilities do you think would impact someone's interpretation of visualization or navigation of the internet, and how? Are there any design considerations you know of that may broaden accessibility? How often do you use them currently?



> The core issue of accessibility is that others might not able to interact with and/or perceiving things in the same exact way as you. I have a mouse but users might be on touch screens, so cannot possibly hover overing something. But even without clicking anything, seeing them alone can already be challenging: color, font size, and whatever causing high cognitive load that make it difficult to understand.
>
> With that said, accessibility needs consideration but not a burden. The key is provide alternatives, can see picture? Provide a text description underneath! I deeply understand the issue as I have some minor color weaknesses (i.e. cannot distinguish between certain colors for being too similar to me), while I still using color, I often using a secondary channels like text or shape to guarantee the understanding.

## After reflection
What did you learn? What was surprising? How might this information change your designs in the future? What are a few concrete changes you can make to make your visualizations more accessible?

> I'm surprised to learn there are way more considerations needed for accessibility, in terms of both breadth and depth. Perception wise it's no longer just can't see the colors but also complete vision loss, the screen reader demo really stands out; as well as interactions with the visualization that it might be impossible to navigate without precise control of mouse. I will put more thoughts on to providing alternative format of my visualizations. For examples, double encodings—never rely on color alone and adding direct labels if possible, meaningful text description for charts.

## Improvement to graph
Then, go into your archive of past homework submissions and choose a visualization to redesign in light of your new knowledge about accessibility.  Explain why, with concrete justification, it is inaccessible. Preserve the original plot, and directly next to it, create a new version that solves these accessibility issues. Be sure to comment on all the aspects of the design or presentation that violate the accessibility guidelines, even if you can't fix them due to our technical limitations.